In [ ]:
import requests
import cv2
import numpy as np
import torch

# Загрузка модели YOLOv5s
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=False)

# Загрузка видео по ссылке
url = "https://www.vidsplay.com/atv-road-aerial/video.mp4"
response = requests.get(url)

# Сохранение содержимого ответа запроса в файл
with open("video.mp4", "wb") as f:
    f.write(response.content)

# Загрузка видео из файла
video = cv2.VideoCapture("video.mp4")

# Установка параметров видео для сохранения
frame_width = int(video.get(3))
frame_height = int(video.get(4))
output_video = cv2.VideoWriter("D:/новая папка/output_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))
# Новое разрешение для кадров
new_width = 640
new_height = 640

# Цикл по кадрам видео
while True:
    # Извлечение кадра из видео
    _, frame = video.read()
    # Проверка, пусто ли изображение
    if frame is not None:
        # Преобразование изображения в формат, подходящий для YOLOv5
        frame = cv2.resize(frame, (640, 640))
        frame = frame.astype(np.float32)

        # Выполнение обнаружения объектов
        detections = model(frame.unsqueeze(0))

        # Вывод отладочных сообщений
        print("Количество обнаруженных объектов:", len(detections))

        # Обработка и отрисовка результатов обнаружения
        for detection in detections:
            if detection["category_id"] in [1, 2, 3, 4, 5]:
                x, y, w, h = detection["bbox"]
                center_x = int(x + w / 2)
                center_y = int(y + h / 2)
                dx = center_x - frame_width // 2
                dy = center_y - frame_height // 2

                # Рисование прямоугольника вокруг объекта
                cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

                # Рисование креста в центре объекта
                cv2.line(frame, (center_x, center_y - 10), (center_x, center_y + 10), (0, 0, 255), 2)
                cv2.line(frame, (center_x - 10, center_y), (center_x + 10, center_y), (0, 0, 255), 2)

                # Вывод смещения на экран
                cv2.putText(frame, f"dX: {dx}, dY: {dy}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Отображение кадра на экран
        cv2.imshow("frame", frame)

        # Запись кадра в выходное видео
        output_video.write(frame)

    # Обработка нажатия клавиши
    key = cv2.waitKey(1)
    if key == 27:
        break

# Закрытие видеофайлов и окон
video.release()
output_video.release()
cv2.destroyAllWindows()


/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 12.4 MB/s eta

KeyboardInterrupt: ignored